In [6]:
%load_ext watermark
%watermark -a "Ruiyu Hu" -d -v -m

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Ruiyu Hu 2019-03-17 

CPython 3.6.7
IPython 7.0.1

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 58 Stepping 9, GenuineIntel
CPU cores  : 4
interpreter: 64bit


In [2]:
# read the pdf file
import PyPDF2 

# tokenize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from textblob import TextBlob

# create inverse index

import math
import glob
import json
import operator
from collections import Counter,defaultdict



import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer


import warnings
warnings.filterwarnings('ignore')

**Create Tokenize**

In [57]:
#%%file tokenize.py
#%%file tokenize.py

from textblob import TextBlob

def get_file_names():
    files = []
    for file in glob.glob("../data/text/*.txt"):
        files.append(file)
    return files
###########################################################################
def convert(file):
    text = []
    with open(str(file), 'rb') as f:
        for line in f.readlines():
            text.append(line.decode("utf-8", "ignore").strip())
    return text

'''def convert_pdf(file):
    pdf_content = []

    pdf = PyPDF2.PdfFileReader(open(str(file),"rb"))
    # pdf may be more than one page
    num_pages = pdf.numPages
    count = 0
    text = ''
    while count < num_pages:
        pageObj = pdf.getPage(count)
        count +=1
        text += pageObj.extractText().replace('\n','')
    if text != '':
        text = text
            
    pdf_content.append(text)   
    return pdf_content'''
###########################################################################     
def tokenize(lst):            
    # create a list of token
    
    tokens = [None] * len(lst)
    for i in range(len(lst)):
        tokens[i] = clean_token(lst[i])
    tokens = [t for tok in tokens for t in tok] 
    return tokens

########################################################################### 
def clean_token(text):
    #porter = nltk.PorterStemmer()
    lemmatizer = nltk.WordNetLemmatizer()
    #snowball = nltk.SnowballStemmer('english')
        
    stopset = set(stopwords.words('english'))
    stopset.update(('less','year'))
    
    noun_lst = []

    for word,tag in (TextBlob(text).tags):
        if tag in ("NN", "NNS", "NNP", "NNPS","JJ"):
            word = word.lower()
            word = lemmatizer.lemmatize(word)
            #word = porter.stem(word)
            #word = snowball.stem(word)
            if word not in stopset and word.isalpha():
                noun_lst.append(word)
    return noun_lst

**create index**

In [58]:
def make_idx(tokens, doc_name, idx, length):
    for term in set(tokens):
        idx[term].append([doc_name,tokens.count(term)])
        length[doc_name] = len(set(tokens))

def write(inverted_idx,len_idx):
    inv_idx_file = open("../data/indexes/inverted_idx.json","w")
    json.dump(inverted_idx,inv_idx_file)

    len_idx_file = open("../data/indexes/len_idx.json","w")
    json.dump(len_idx,len_idx_file)
    
def generate_idx():
    resume_files = get_file_names()
    inverted_index = defaultdict(list)
    length_index = defaultdict(list)
    
    for file in resume_files:
        make_idx(tokenize(convert(file)), file, inverted_index, length_index)
        
    write(inverted_index,length_index)
    print ("Indexes generated")
    

In [59]:
generate_idx()

Indexes generated


In [ ]:
'''inv_idx_file = open("../data/indexes/inverted_idx.json","r")
inv_indx = json.load(inv_idx_file)
# create dictionary
doc_freq ={}
for key in sorted(inv_indx.keys()):
    doc_freq[key] = sum(Counter(set(doc_id for doc_id, term in inv_indx[key])).values())

dictionary = pd.DataFrame.from_dict(doc_freq,orient='index', columns=['DocFreq'])
dictionary.head()'''

**create retrieval-The BM25 Weighting Scheme**

In [60]:
from math import log

'''
IR Book: 11.4.3
Fomula: 11.33
'''
'''
typical TREC value (Text Retrieval Conference (TREC).)
f1 = 1.2
k2 varies from 0 to 1000
b = 0.75
'''

k1 = 1.2
b = 0.75
k2 = 100
R = 0 # (set it to 0 since no relevancy info is known)



def BM25(doc_len, avg_doc_len, n_doc_w_term, n_total_doc, freq_term_doc, freq_term_query, rel_doc_w_term):
    
    n = n_doc_w_term
    N = n_total_doc
    f = freq_term_doc
    q = freq_term_query
    r = rel_doc_w_term
    
    p1 = ((k2 + 1) * q) / (k2 + q) #Relevance between term and query
    p2 = ((k1 + 1) * f) / (getK(doc_len, avg_doc_len) + f) #Relevance between term and document
    p3 = log((((r + 0.5)/(R-r+0.5)) / ((n - r + 0.5)/(N - n - R + r + 0.5)))+1) # Term Weight
    return p1 * p2 * p3

def getK(doc_len, avg_doc_len):
    return k1 * ((1 - b) + b * (float(doc_len) / float(avg_doc_len)))
    
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [99]:
# get average document length
def get_avg_doc_len(len_idx):
    _length = 0
    for doc in len_idx:
        _length += len_idx[doc]
    return float(_length) / float(len(len_idx))

def search(query):
    inv_idx_file = open("../data/indexes/inverted_idx.json","r")
    inverted_idx = json.load(inv_idx_file)

    len_idx_file = open("../data/indexes/len_idx.json","r")
    len_idx = json.load(len_idx_file)

    scores = defaultdict(list)
    
    query_tokens = query.split()
    for token in query_tokens:
    #for token in query:
        #token = token.lower()
        for tok in clean_token(token):
            if tok in inverted_idx.keys():
                for entry in inverted_idx[tok]:
                    bm25_val = BM25(len_idx[entry[0]],get_avg_doc_len(len_idx),len(inverted_idx[tok]),len(len_idx),entry[1],1,0)
                #scores[entry[0]] = round(10* sigmoid(bm25_val)-5,4)
                    scores[entry[0]] = round(bm25_val,4)
    result = sorted(scores.items(),key=operator.itemgetter(1),reverse=True)
    return result

In [102]:
query = 'JAVA'
search(query)

[('../data/text\\106750694Siva Resume.txt', 4.2557),
 ('../data/text\\114314467viswanadhCV.txt', 3.3382)]

**w2v approach**

In [10]:
import time
from gensim.models import KeyedVectors
t1 = time.time()
#download link: https://github.com/mmihaltz/word2vec-GoogleNews-vectors
path = 'C:/Users/RayHu/Downloads/google_w2v/GoogleNews-vectors-negative300.bin'
w2v_model = KeyedVectors.load_word2vec_format(path, binary=True)
print('-------------------------------------------')
print("Loading word2vec model cost %.3f seconds...\n" % (time.time() - t1))

# -------------------------------------------
#Loading word2vec model cost 108.422 seconds...

-------------------------------------------
Loading word2vec model cost 93.678 seconds...



In [49]:
#%%file vectorize.py
import numpy as np
from numpy import linalg as LA

def vectorize(words):
    '''
    transform the doc and query into vectors
    '''
    word_vec = []
    for word in words:
        try:
            vec = w2v_model[word]
            word_vec.append(vec)
        except KeyError:
            # ignore the word if it is not in the w2v vocablary
            pass
    vector = np.mean(word_vec,axis = 0)
    return vector

def cos_sim(vector1, vector2):
    '''
    the fomula to calculte cosine cimilarity 
    '''
    sim = np.dot(vector1, vector2) / (LA.norm(vector1) * LA.norm(vector2))
    
    if np.isnan(np.sum(sim)):
        return 0
    
    return sim

def calc_sim(query):
    '''
    calculate similarity scores between documents and the query
    '''
    query = clean_token(query)
    file_list = get_file_names()
    documents = {}
    
    for i in range(len(file_list)):
        documents[file_list[i]] = tokenize(convert(file_list[i]))
        
    query_vec = vectorize(query)
    results = {}
    
    for name, doc in documents.items():
        doc_vec = vectorize(doc)
        sim_score = cos_sim(query_vec, doc_vec)
        #threshold = 0.55
        if sim_score > 0:
            results[name] = sim_score
            sort_result = sorted(results.items(),key=operator.itemgetter(1),reverse=True)
    return sort_result

**jaccard**

In [70]:
def jaccard(query, documents):
    s1 = set(query)
    s2 = set(documents)
    lst1 = s1.intersection(s2)
    lst2 = s1.union(s2)
    jaccard = 1.0 * len(lst1)/len(lst2)
    
    return jaccard

def calc_jac(query):
    query = set(clean_token(query))
    
    file_list = get_file_names()
    documents = {}
    
    for i in range(len(file_list)):
        documents[file_list[i]] = set(tokenize(convert(file_list[i])))
    
    results = {}
    
    for name, doc in documents.items():
        jac = jaccard(query, doc)
        if jac > 0:
            results[name] = round(jac,4)
            sort_result = sorted(results.items(),key=operator.itemgetter(1),reverse=True)
    return sort_result

**Search**

In [74]:
def matching(query):
    #print(query)
    print('-------------------------------------------------------')
    t1 = time.time()
    results_bm25 = search(query)[:5]
    print('bm25 result')
    for result in results_bm25:
        print(result)
    print('The computing cost %.3f seconds\n'% (time.time() - t1))
    print('-------------------------------------------------------')
    t2 = time.time()
    results_cos = calc_sim(query)[:5]
    print('cosine result')
    for result in results_cos:
        print(result)
    print('The computing cost %.3f seconds\n'% (time.time() - t2))   
    print('-------------------------------------------------------')
    t3 = time.time()
    results_jac = calc_jac(query)[:5]
    print('jacarrd result')
    for result in results_jac:
        print(result)
    print('The computing cost %.3f seconds\n'% (time.time() - t3))
    print('-------------------------------------------------------')

In [75]:
matching('java')

-------------------------------------------------------
bm25 result
('../data/text\\106750694Siva Resume.txt', 4.2557)
('../data/text\\114314467viswanadhCV.txt', 3.3382)
The computing cost 0.024 seconds

-------------------------------------------------------
cosine result
('../data/text\\105903931resume__Atanu.txt', 0.33573505)
('../data/text\\114314467viswanadhCV.txt', 0.32138988)
('../data/text\\103645689naushad new final.txt', 0.30411604)
('../data/text\\106750694Siva Resume.txt', 0.29796073)
('../data/text\\107279656Copy_of_Water_soluble_oils.txt', 0.2946744)
The computing cost 16.804 seconds

-------------------------------------------------------
jacarrd result
('../data/text\\106750694Siva Resume.txt', 0.0111)
('../data/text\\114314467viswanadhCV.txt', 0.0054)
The computing cost 17.496 seconds

-------------------------------------------------------


In [ ]:
#matching('define business metric, perform insight analysis')

**auto test**

In [142]:
lst = ['sql or Java', 'electrical', 'database', 'cad', 'plumber',
       'electronics', 'mechanical', 'mba', 'safety', 'java', 'marketing',
       'matlab', 'assistant', 'construction', 'finance', 'finance mba',
       'marketing mba', 'sales', 'construction engineer', 'telecom', 'office']

In [143]:
import pandas as pd
def auto_test():
    df = pd.read_csv('testset_new.csv')
    dict1 = {}
    dict2 = {}
    for i in lst:
        dict1[i] = len(df[df[i]=='R'])
        dict2[i] = len(calc_jac(i))
                  
    pd1 = pd.DataFrame.from_dict(dict1, orient='index')
    pd2 = pd.DataFrame.from_dict(dict2, orient='index')
    return pd.concat([pd1,pd2], axis = 1)

In [144]:
auto_test().to_csv('test_result_jac.csv')